In [59]:
from os import listdir
from pathlib import Path
import json

import torch
import random
from torch.utils.data import DataLoader,Dataset
from PIL import Image
from torchvision import transforms

# CLIP 官方预训练时常用的归一化参数（RGB 三个通道的均值和标准差）
CLIP_MEAN = [0.48145466, 0.4578275, 0.40821073]
CLIP_STD  = [0.26862954, 0.26130258, 0.27577711]
class WenwuDataset:
    def __init__(self,data_json_path:str="/Users/zsh2401/Lab/Spiders/wenwu/data.json",images_path:str="/Users/zsh2401/Lab/Spiders/wenwu/output_root"):
        with open(data_json_path,'r') as f:
            self.data = json.load(f)["items"]
        self.ids = sorted(self.data.keys())
        self.images_root = Path(images_path)
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop((224,224),scale=(0.8,1.0),ratio=(3/4,4/3)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(CLIP_MEAN,CLIP_STD)
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        uuid = self.ids[idx]
        item =  self.data[uuid]
        item_image_folder = self.images_root / uuid[0:2] / uuid[2:4] / uuid
        images = []
        for name in listdir(item_image_folder):
            fpath = item_image_folder / name
            image = Image.open(fpath).convert('RGB')
            # image = self.transform(image)
            images.append(image)
        text_pos = f'{item["name"]},{item['meta']['年代']},{",".join(item["types"])}'
        text_neg = f'{item["name"]}'
        return images[0],text_pos,text_neg

dataset = WenwuDataset()
dataset[0],len(dataset)

((<PIL.Image.Image image mode=RGB size=800x600>,
  '青釉刻莲瓣纹钵,隋,钵（容器）,瓷钵,瓷（陶瓷）,青瓷,青釉钵,陶瓷,颜色釉瓷',
  '青釉刻莲瓣纹钵'),
 7673)

In [62]:
# train_wenwu_clip.py
# ----------------------------------------------------------
# 0. 可配置超参
BATCH_SIZE      = 16          # 128 for 24 GB, 256 for 40 GB
EPOCHS          = 5
LR              = 5e-6
FREEZE_VIT_LAYERS = 10         # 冻结前 10 层视觉 backbone
FREEZE_TXT_LAYERS = 9          # 同理文本
MODEL_NAME      = "openai/clip-vit-base-patch32"   # 或 hfl/chinese-clip-vit-b32
DATA_PATH       = "data/wenwu_meta.jsonl"
OUTPUT_DIR      = "wenwu-clip-vit-b32"
# ----------------------------------------------------------

import json, random, os, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel, get_linear_schedule_with_warmup

device = torch.device("mps")
torch.backends.cuda.matmul.allow_tf32 = True  # 加速

# 2. collate：一次性把正负喂进 processor
def collate_fn(batch):
    imgs, txt_pos, txt_neg = zip(*batch)
    processor =  CLIPProcessor.from_pretrained(MODEL_NAME)
    # 拼 batch：图像重复两次，文本 = 正 + 负
    imgs2  = imgs + imgs                      # 2*B
    texts2 = list(txt_pos) + list(txt_neg)    # 2*B
    inputs = processor(text=texts2, images=imgs2,
                       truncation=True,
                       return_tensors="pt", padding=True)
    return inputs

# 3. 准备模型并冻结部分层
model = CLIPModel.from_pretrained(MODEL_NAME)
# 冻结视觉 backbone 前 FREEZE_VIT_LAYERS 层
for name, param in model.vision_model.named_parameters():
    if name.split(".")[1].isdigit() and int(name.split(".")[1]) < FREEZE_VIT_LAYERS:
        param.requires_grad_(False)
# 冻结文本编码器前 FREEZE_TXT_LAYERS 层
for name, param in model.text_model.named_parameters():
    if name.startswith("encoder.layers"):
        layer_idx = int(name.split(".")[2])
        if layer_idx < FREEZE_TXT_LAYERS:
            param.requires_grad_(False)
model = model.to(device)

# InfoNCE = 交叉熵(相似度矩阵 / τ)，直接用 CLIP 的 loss
criterion = nn.CrossEntropyLoss()

# 4. DataLoader & 优化器
dl  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,collate_fn=collate_fn, pin_memory=True)

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad,
                                     model.parameters()), lr=LR, weight_decay=0.2)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=len(dl)*1, num_training_steps=len(dl)*EPOCHS)

# 5. 训练循环
global_step = 0
model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(dl, desc=f"epoch {epoch+1}/{EPOCHS}")
    for batch in pbar:

        batch = {k:v.to(device) for k,v in batch.items()}
        print("what")
        # forward
        outputs = model(**batch)
        img_emb = outputs.image_embeds            # shape (2B, D)
        txt_emb = outputs.text_embeds

        # 构造对比标签：
        # seq: [正 img1, img1_neg]  [正 img2, img2_neg] ...
        # 正确对是 idx_i == idx_(i+B)
        logits = img_emb @ txt_emb.t()            # 2B x 2B
        labels = torch.arange(len(img_emb), device=device)
        loss_i2t = criterion(logits / 0.01, labels)            # img→text
        loss_t2i = criterion(logits.t() / 0.01, labels)        # text→img
        loss = (loss_i2t + loss_t2i)/2

        loss.backward()
        optimizer.step(); scheduler.step()
        optimizer.zero_grad(set_to_none=True)

        pbar.set_postfix(loss=float(loss), lr=scheduler.get_last_lr()[0])
        global_step += 1

    # 可选：每 epoch 保存 ckpt
    model.save_pretrained(f"{OUTPUT_DIR}/ckpt_epoch{epoch+1}")

# 6. 训练完成后保存最终权重
model.save_pretrained(OUTPUT_DIR)
collate_fn.processor.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

epoch 1/5:   0%|          | 0/480 [00:00<?, ?it/s]

what


epoch 1/5:   0%|          | 1/480 [00:10<1:27:14, 10.93s/it, loss=4.49, lr=1.04e-8]

what


epoch 1/5:   0%|          | 2/480 [00:16<1:00:08,  7.55s/it, loss=5.38, lr=2.08e-8]

...

epoch 1/5:  46%|████▌     | 221/480 [23:15<31:02,  7.19s/it, loss=2.4, lr=2.3e-6]  

what


epoch 1/5:  46%|████▋     | 222/480 [23:21<30:03,  6.99s/it, loss=2.5, lr=2.31e-6]

what


epoch 1/5:  46%|████▋     | 223/480 [23:33<27:08,  6.34s/it, loss=2.32, lr=2.32e-6]

KeyboardInterrupt

